# Ingestion des données en base de données

Pour résumer, l'objectif du projet est de s'entraîner en SQL et en Python pour se maintenir à jour en SQL (notamment les CTEs, les windows functions et les RANKS) et en Python (dataframes, SQLAlchemy, Pydantic, FastAPI, tests unitaires, Mock si pertinent, et si pertinent un peu d'intégration de modèles de machine learning).

La première chose dans ce notebook ça va être de préparer les données et de les importer dans une DB PostgreSQL.


## Cleaning data

D'abord on regarde les données pour comprendre la structure.

In [ ]:
import pandas as pd
import numpy as np

dataset_path = "Olympic_Swimming_Results_1912to2020"

# Lire le fichier CSV et afficher les colonnes
df = pd.read_csv(dataset_path + ".csv")
print(df.columns)
print(df.head())

distinct_distances = df['Distance (in meters)'].unique()  # Assurez-vous que le nom de la colonne est correct
print("Distances distinctes:", distinct_distances)

distinct_results = df['Results'].unique()
print("Nombre de résultats distincts:", len(distinct_results))

# Sélectionner aléatoirement 100 résultats distincts (ou moins s'il y en a moins de 100)
sample_size = min(100, len(distinct_results))
random_sample = np.random.choice(distinct_results, size=sample_size, replace=False)

print(f"\n{sample_size} résultats distincts aléatoires:")
for result in random_sample:
    print(result)

On renomme les colonnes pour les rendre plus faciles à utiliser.

In [ ]:
import pandas as pd
import re

# Renommer les colonnes
df = df.rename(columns={
    'Location': 'location',
    'Year': 'year',
    'Distance (in meters)': 'distance',
    'Stroke': 'stroke',
    'Relay?': 'is_relay',
    'Gender': 'gender',
    'Team': 'team',
    'Athlete': 'athlete',
    'Results': 'results',
    'Rank': 'rank'
})

print(df.head())

On convertit la colonne is_relay en booléen.

In [ ]:
# Convertir 'is_relay' en booléen
df['is_relay'] = df['is_relay'].map(lambda x: True if x == 1 else False)

print(df.head())

On extrait la distance et le nombre de relais. On aura besoin de la distance en entier pour analyser correctement les données. Dans le CSV de base c'est un string au format [0-9]+m si c'est pas du relais (e.g 100m), et [0-9]+x[0-9]+m si c'est un relais (e.g 4x100m).

In [ ]:
import re

# Fonction pour extraire la distance et le nombre de relais
def extract_distance_and_relay(distance_str):
    if 'x' in distance_str:
        nb_relay_str, distance_str = re.findall(r'\d+', distance_str)
        print(nb_relay_str, distance_str)
        return int(distance_str), int(nb_relay_str)
    else:
        distance = int(re.findall(r'\d+', distance_str)[0])
        return distance, None

# Appliquer la transformation à la colonne 'distance'
df['distance'], df['nb_relay'] = zip(*df['distance'].apply(extract_distance_and_relay))

print(df.head())

df_relay_true = df[df['is_relay'] == True]
print(df_relay_true.head())

Ici, faut harmoniser les formats de temps. Dans le CSV de base, les temps sont dans des formats divers, parfois avec des minutes et des secondes, parfois avec des heures, parfois avec des décimales. On va tout convertir en secondes, sous forme de float avec une précision à la microseconde.

In [ ]:
import re
import numpy as np

def clean_time_str(time_str):
    if isinstance(time_str, str):
        # Extraire les chiffres et les séparateurs pertinents
        cleaned = re.match(r'^(\d+:?\d*:?\d*\.?\d*)', time_str)
        if cleaned:
            return cleaned.group(1)
        elif not re.search(r'\d', time_str):
            return time_str  # Retourner la chaîne si elle ne contient aucun chiffre
    return time_str

def convert_to_seconds(time_str):
    original_value = time_str
    time_str = clean_time_str(time_str)
    
    if isinstance(time_str, float):
        return time_str, None  # Déjà en secondes
    elif isinstance(time_str, str):
        if ':' in time_str:
            # Format 00:04:37.510000
            time_parts = time_str.split(':')
            if len(time_parts) == 3:
                hours, minutes, seconds = time_parts
                total_seconds = int(hours) * 3600 + int(minutes) * 60 + float(seconds)
            else:
                minutes, seconds = time_parts
                total_seconds = int(minutes) * 60 + float(seconds)
        elif re.match(r'^\d+(\.\d+)?$', time_str):
            # Format 59.720
            total_seconds = float(time_str)
        else:
            return np.nan, original_value  # Cas de disqualification ou format invalide
    else:
        return np.nan, str(original_value)  # Cas où le type n'est pas reconnu
    
    return round(total_seconds, 3), None  # Arrondir à 3 décimales pour la précision milliseconde

# Appliquer la conversion à la colonne 'results'
df['results'], df['quit_reason'] = zip(*df['results'].apply(convert_to_seconds))

# Afficher les premières lignes pour vérification
print(df.head())
print("\nColonnes du DataFrame:", df.columns)

# Afficher 50 valeurs aléatoires du DataFrame
random_sample = df.sample(n=50, random_state=1)  # random_state pour la reproductibilité
print("\n50 valeurs aléatoires du DataFrame :")
print(random_sample)